# Processing an eDNA dataset to Darwin Core

## List all dataset files

In [1]:
list.files("../dataset", full.names = "TRUE")

[1] "../dataset/metadata.txt"    "../dataset/samples.xlsx"   
[3] "../dataset/seqtab.txt"      "../dataset/sequences.fasta"
[5] "../dataset/taxonomy.txt"

## Read the ASV table

`../dataset/seqtab.txt` contains the ASV table, so it has one row per ASV, and the number of reads in a sample in different columns.

In [2]:
library(dplyr)

seqtab <- read.table("../dataset/seqtab.txt", sep = "\t", header = TRUE)
head(seqtab)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




,asv,EE0493,EE0495
,<chr>,<int>,<int>
1,asv.1,0,0
2,asv.2,14,2447
3,asv.3,0,0
4,asv.4,0,0
5,asv.5,40587,1857
6,asv.6,7,10


## Read the taxonomy file

`../dataset/taxonomy.txt` contains a taxon name for each ASV.

In [3]:
taxonomy <- read.table("../dataset/taxonomy.txt", sep = "\t", header = TRUE)
head(taxonomy)

,asv,taxonomy
,<chr>,<chr>
1,asv.1,Eukaryota
2,asv.2,Clausocalanus_furcatus
3,asv.3,Eurotatoria
4,asv.4,Arthropoda
5,asv.5,Eukaryota
6,asv.6,Farranula_gibbula


These names originate from the reference database and will have to be matched to WoRMS later.

In [4]:
## Read the sample metadata

We also have an Excel file with sample info.

In [7]:
samples <- readxl::read_excel("../dataset/samples.xlsx")
samples

name,size,event_begin,area_name,area_longitude,area_latitude,area_uncertainty,parent_area_name,dna
<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>
EE0493,1450,24/04/2023,Ile esprit,46.22536,9.425180,20,Aldabra Atoll,7.23
EE0495,1500,02/04/2023,Settlement beach,46.20605,9.400901,98,Aldabra Atoll,15.83
